In [8]:
import math
import random

In [2]:
def forward_multiply_gate(x,y):
    return x * y 

forward_multiply_gate(-2,3)  # returns -6

-6

In [10]:
# circuit with a single gate

def forward_multiply_gate(x,y): 
    return x *y

x, y = -2,3 # input values

# try changing x,y randomly small amounts and keep track of what works best
tweak_amount = 0.01
best_out = -math.inf
best_x, best_y = x, y

for i in range(100):
    x_try = x + tweak_amount * (random.random() * 2 - 1) # tweak x a little bit
    y_try = y + tweak_amount * (random.random() * 2 - 1) # tweak y a little bit
    out = forward_multiply_gate(x_try,y_try)
    if out > best_out:
        best_out = out
        best_x, best_y = x_try, y_try

In [14]:
print("best x: {}".format(best_x))
print("best y: {}".format(best_y))
print("best out: {}".format(best_out))

best x: -1.9906748071853988
best y: 2.990304691028108
best out: -5.952724214237973


In [19]:
x, y = -2, 3
out = forward_multiply_gate(x,y)
h = 0.0001

# compute derivative with respect to x
xph = x + h
out2 = forward_multiply_gate(xph, y) # -5.997
x_derivative = (out2 - out) / h # 3.0

# compute derivative with respect to y
yph = y + h
out3 = forward_multiply_gate(x, yph) # -6.0002
y_derivative = (out3 - out)/h # 2.0

In [20]:
y_derivative

-2.0000000000042206

In [91]:
class Unit:
    def __init__(self, value, grad):
        self.value = value
        self.grad = grad
        


In [92]:
class MultiplyGate:
    def forward(self,u0,u1):
        self.u0 = u0
        self.u1 = u1
        self.utop = Unit(u0.value*u1.value, 0.0)
        return self.utop
    
    def backward(self):
        self.u0.grad += self.u1.value * self.utop.grad
        self.u1.grad += self.u0.value * self.utop.grad

In [93]:
class AddGate:
    def forward(self,u0,u1):
        self.u0 = u0
        self.u1 = u1
        self.utop = Unit(u0.value + u1.value, 0.0)
        return self.utop
    
    def backward(self):
        self.u0.grad += 1 * self.utop.grad
        self.u1.grad += 1 * self.utop.grad

In [122]:
# helper function
def sigmoid_fun(x):
    return 1/(1+math.exp(-x))

class SigmoidGate:

    
    def forward(self,u0):
        self.u0 = u0
        self.utop = Unit(sigmoid_fun(self.u0.value), 0.0)
        return self.utop
    
    def backward(self):
        s = sigmoid_fun(self.u0.value)
        self.u0.grad += (s * (1 - s)) * self.utop.grad
    

In [123]:
a = Unit(1.0,0.0)
b = Unit(2.0,0.0)
c = Unit(-3.0, 0.0)
x = Unit(-1.0,0.0)
y = Unit(3.0,0.0)

In [124]:
mulg0 = MultiplyGate()
mulg1 = MultiplyGate()
addg0 = AddGate()
addg1 = AddGate()
sg0 = SigmoidGate()

In [125]:


ax = mulg0.forward(a,x)
by = mulg1.forward(b,y)
axpby = addg0.forward(ax,by)
axpbypc = addg1.forward(axpby,c)
s = sg0.forward(axpbypc)


In [126]:
s.value

0.8807970779778823

In [127]:
s.grad = 1.0

In [128]:
sg0.backward()
addg1.backward()
addg0.backward()
mulg1.backward()
mulg0.backward()

In [129]:
a.grad

-0.10499358540350662

In [130]:
step_size = 0.01

a.value += step_size * a.grad
b.value += step_size * b.grad
c.value += step_size * c.grad
x.value += step_size * x.grad
y.value += step_size * y.grad

In [131]:
ax = mulg0.forward(a,x)
by = mulg1.forward(b,y)
axpby = addg0.forward(ax,by)
axpbypc = addg1.forward(axpby,c)
s = sg0.forward(axpbypc)

In [132]:
s.value

0.8825501816218984

In [133]:
def forward_fast_circuit(a,b,c,x,y):
    return 1/(1 + math.exp(- (a*x + b*y + c)))

In [134]:
a,b,c,x,y = 1,2,-3,-1,3
h = 0.0001

In [135]:
a_grad = (forward_fast_circuit(a+h,b,c,x,y) - forward_fast_circuit(a,b,c,x,y))/h
b_grad = (forward_fast_circuit(a,b+h,c,x,y) - forward_fast_circuit(a,b,c,x,y))/h
c_grad = (forward_fast_circuit(a,b,c+h,x,y) - forward_fast_circuit(a,b,c,x,y))/h
x_grad = (forward_fast_circuit(a,b,c,x+h,y) - forward_fast_circuit(a,b,c,x,y))/h
y_grad = (forward_fast_circuit(a,b,c,x,y+h) - forward_fast_circuit(a,b,c,x,y))/h

In [137]:
print(a_grad,b_grad,c_grad,x_grad,y_grad)

-0.10499758359205913 0.3149447748351797 0.10498958734506125 0.10498958734506125 0.2099711788272618
